# Comparing FD and FEM
Below is a (very) simple case of a string (or rod) with equation
$$ \frac{\partial^2 u}{\partial x^2} + f(x) = 0 $$
on the domain $0$ to $L$, with 4 equally spaced nodes $x_1$ to $x_4$.  The nodes are spaced $h$ apart.  The force $f$ is also discretized at each node $f_1$ to $f_4$.  At the boundaries we have the Dirchlet conditions
$$ u(x_1) = g_1,\quad u(x_2) = g_2 $$

The goal of this sheet is to explore the finite-difference scheme and finite element method on this toy problem.  They primarily give the same same results, but it is easier to account for variable $f(x)$ with the finite element method.

In [1]:
using Pkg
Pkg.activate(".")
using SymPy

 Activating environment at `C:\Users\tim\Documents\ensc481-2020-examples\class-demos\04-16\Project.toml`
┌ Info: Precompiling SymPy [24249f21-da20-56a4-8eb1-6a02cf4ae2e6]
└ @ Base loading.jl:1260


In [2]:
@vars u1 u2 u3 u4 x1 x2 x3 x4 f1 f2 f3 f4 g1 g2 h

(u1, u2, u3, u4, x1, x2, x3, x4, f1, f2, f3, f4, g1, g2, h)

## Finite Differences

In [3]:
eq1 = Eq( u1, g1 )

u1 = g1

In [4]:
eq2 = Eq( (u3 - 2u2 + u1)/h^2 + f2, 0 )

     u1 - 2*u2 + u3    
f2 + -------------- = 0
            2          
           h           

In [5]:
eq3 = Eq( (u4 - 2u3 + u2)/h^2 + f3, 0)

     u2 - 2*u3 + u4    
f3 + -------------- = 0
            2          
           h           

In [6]:
eq4 = Eq( u4, g2 )

u4 = g2

In [7]:
solve([eq1,eq2,eq3,eq4], [u1,u2,u3,u4] )

Dict{Any,Any} with 4 entries:
  u1 => g1
  u2 => 2*f2*h^2/3 + f3*h^2/3 + 2*g1/3 + g2/3
  u4 => g2
  u3 => f2*h^2/3 + 2*f3*h^2/3 + g1/3 + 2*g2/3

## Finite Element Method
using linear 1D 2-node elements

In [8]:
@vars δq1 δq2 δq3 δq4
ξ = symbols("xi", real=true)
;

In [9]:
N = [ (1-ξ)//2 (1+ξ)//2 ]

1×2 Array{Sym,2}:
 1/2 - xi/2  xi/2 + 1/2

In [10]:
# ke(x1,x2) = 1/(x2-x1)*[1 -1; -1 1]
ke = 1/h*[1 -1; -1 1]

2×2 Array{Sym,2}:
  1/h  -1/h
 -1/h   1/h

In [11]:
segment(q1, q2,δq1,δq2) =  -([δq1 δq2]*ke*[q1,q2] )[1]

segment (generic function with 1 method)

Let's handle the forcing two different ways.  Method 1: interpolate between the nodes
$$ f^e(x) =  N(\xi) \left[\begin{array}{x}  f_1^e \\ f_2^e \end{array} \right]$$
Then we need to evaluate (for each element)
$$ \delta q^T \int\limits_{-1}^{1} N^T f(x)\, dx = 
\delta q^T \int\limits_{-1}^{1} N^T N(\xi) \left[\begin{array}{x}  f_1^e \\ f_2^e \end{array} \right] \dfrac{h}{2} \, d\xi $$

In [12]:
forcesegment(f1,f2,δq1,δq2) = ( [δq1 δq2]*[ integrate( a, (ξ,-1,1) ) |> simplify for a in transpose(N)*(N*[f1,f2])*h//2 ] )[1]

forcesegment (generic function with 1 method)

Now let's build the entire equation

In [13]:
fem1a = segment(u1,u2, δq1, δq2) + segment(u2,u3, δq2, δq3) + segment(u3,u4, δq3, δq4) + forcesegment(f1,f2,δq1,δq2) + forcesegment(f2,f3,δq2,δq3) + forcesegment(f3,f4,δq3,δq4)

h*δq1*(2*f1 + f2)   h*δq2*(f1 + 2*f2)   h*δq2*(2*f2 + f3)   h*δq3*(f2 + 2*f3) 
----------------- + ----------------- + ----------------- + ----------------- 
        6                   6                   6                   6         

  h*δq3*(2*f3 + f4)   h*δq4*(f3 + 2*f4)      /δq1   δq2\      /  δq1   δq2\   
+ ----------------- + ----------------- - u1*|--- - ---| - u2*|- --- + ---| - 
          6                   6              \ h     h /      \   h     h /   

   /δq2   δq3\      /  δq2   δq3\      /δq3   δq4\      /  δq3   δq4\
u2*|--- - ---| - u3*|- --- + ---| - u3*|--- - ---| - u4*|- --- + ---|
   \ h     h /      \   h     h /      \ h     h /      \   h     h /

Taking only the variations that are free to move, and extracting their coefficients gives

In [14]:
fem1b = [ diff(fem1a, δq) for δq in [δq2,δq3] ]

2-element Array{Sym,1}:
 h*(f1 + 2*f2)/6 + h*(2*f2 + f3)/6 + u1/h - 2*u2/h + u3/h
 h*(f2 + 2*f3)/6 + h*(2*f3 + f4)/6 + u2/h - 2*u3/h + u4/h

Appending the boundary essential conditions

In [15]:
fem1c = vcat( fem1b, [u1-g1, u4-g2] )

4-element Array{Sym,1}:
 h*(f1 + 2*f2)/6 + h*(2*f2 + f3)/6 + u1/h - 2*u2/h + u3/h
 h*(f2 + 2*f3)/6 + h*(2*f3 + f4)/6 + u2/h - 2*u3/h + u4/h
                                                 -g1 + u1
                                                 -g2 + u4

and solving the displacements

In [16]:
solve(fem1c, [u1,u2,u3,u4] )

Dict{Any,Any} with 4 entries:
  u1 => g1
  u2 => f1*h^2/9 + f2*h^2/2 + f3*h^2/3 + f4*h^2/18 + 2*g1/3 + g2/3
  u4 => g2
  u3 => f1*h^2/18 + f2*h^2/3 + f3*h^2/2 + f4*h^2/9 + g1/3 + 2*g2/3

These equations (while a solution) don't match the FD from before.  So we'll make a change to how we handle $f$.  Instead of treating $f$ as a piecewise-linear function, let's assume that it is only acting directly at the node
$$ f^e(x) =  f_1 \delta\left( \xi + 1 \right)  + f_2 \delta\left( \xi - 1 \right)$$
Now we need to evaluate 
$$ \delta q^T \int\limits_{-1}^{1} N^T f(x)\, dx = \delta q^T \int\limits_{-1}^{1} N^T \left(  f_1 \delta\left( \xi + 1 \right)  + f_2 \delta\left( \xi - 1 \right) \right)\frac{h}{2}\, d\xi =
\delta q^T \left( N^T(-1)f_1 + N^T(1)f_2 \right) \frac{h}{2} $$

In [17]:
forcesegment_alt(f1,f2,δq1,δq2)  = ( [δq1 δq2]*( [ subs(n,ξ=>-1) for n in transpose(N) ]*f1*h//2 +  [ subs(n,ξ=>+1) for n in transpose(N) ]*f2*h/2 ) )[1]

forcesegment_alt (generic function with 1 method)

In [18]:
fem2a = segment(u1,u2, δq1, δq2) + segment(u2,u3, δq2, δq3) + segment(u3,u4, δq3, δq4) + forcesegment_alt(f1,f2,δq1,δq2) + forcesegment_alt(f2,f3,δq2,δq3) + forcesegment_alt(f3,f4,δq3,δq4)

f1*h*δq1                         f4*h*δq4      /δq1   δq2\      /  δq1   δq2\ 
-------- + f2*h*δq2 + f3*h*δq3 + -------- - u1*|--- - ---| - u2*|- --- + ---| 
   2                                2          \ h     h /      \   h     h / 

     /δq2   δq3\      /  δq2   δq3\      /δq3   δq4\      /  δq3   δq4\
- u2*|--- - ---| - u3*|- --- + ---| - u3*|--- - ---| - u4*|- --- + ---|
     \ h     h /      \   h     h /      \ h     h /      \   h     h /

In [19]:
fem2b = [ diff(fem2a, δq) for δq in [δq2,δq3] ]

2-element Array{Sym,1}:
 f2*h + u1/h - 2*u2/h + u3/h
 f3*h + u2/h - 2*u3/h + u4/h

In [20]:
fem2c = vcat( fem2b, [u1-g1, u4-g2] )

4-element Array{Sym,1}:
 f2*h + u1/h - 2*u2/h + u3/h
 f3*h + u2/h - 2*u3/h + u4/h
                    -g1 + u1
                    -g2 + u4

In [21]:
solve(fem2c, [u1,u2,u3,u4] )

Dict{Any,Any} with 4 entries:
  u1 => g1
  u2 => 2*f2*h^2/3 + f3*h^2/3 + 2*g1/3 + g2/3
  u4 => g2
  u3 => f2*h^2/3 + 2*f3*h^2/3 + g1/3 + 2*g2/3

Now these match the FD version of the solution.  A lesson from the comparison here is that we can consistently handle complicated loading.  While we could do this in the FD method, it is less commonly done and may become very complicated in higher order/higher dimensioned problems.